<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/Pivot_Tables_for_OLAP_Cube_Python_Superstore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

In [ ]:
from datetime import datetime
import pytz
print('ॐ श्री सरस्वत्यै नमः',datetime.now(pytz.timezone('Asia/Calcutta')))
!python --version
!lsb_release -a

ॐ श्री सरस्वत्यै नमः 2023-09-11 05:56:07.396593+05:30
Python 3.10.12
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.2 LTS
Release:	22.04
Codename:	jammy


For a more detailed exploration of Pivot Tables please look at this [Notebook : Analysis of Retail Sales Data](https://github.com/Praxis-QR/PythonForManagers/blob/main/Retail_Sales_Data_Analysis.ipynb)

#SuperStore
Super Store Order Data https://drive.google.com/file/d/13MKOpo8iD-tMkeMLw-4IPFi5-xWhmqFZ/view?usp=sharing <br>
Original Data from [Tableau](https://community.tableau.com/docs/DOC-1236)

In [ ]:
#!wget -O SS_Orders2.csv 'https://drive.google.com/uc?export=download&id=13MKOpo8iD-tMkeMLw-4IPFi5-xWhmqFZ'
#!gdown https://drive.google.com/uc?id=13MKOpo8iD-tMkeMLw-4IPFi5-xWhmqFZ
!wget -q -O SS_Orders.csv 'https://raw.githubusercontent.com/Praxis-QR/RDWH/main/data/SS_Orders.csv'

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
dfSS = pd.read_csv('SS_Orders.csv', delimiter = ',')
dfSS.head(2)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
1,2,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820


In [ ]:
dfSS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

# CrossTab | Pivot | Brute Force

Pivot <br>
https://dfrieds.com/data-analysis/pivot-table-python-pandas.html

In [ ]:
#Create the pivot table
dfSSPivot = pd.pivot_table(data=dfSS,index='Segment',columns='Category',values='Sales', aggfunc='sum').copy()
dfSSPivot

Category,Furniture,Office Supplies,Technology
Segment,,,
Consumer,391049.3120,363952.136,406399.897
Corporate,229019.7858,230676.462,246450.119
Home Office,121930.6975,124418.434,183304.017


In [ ]:
#Calculate the Row Totals
dfSSPivot['RowTotals'] = (dfSSPivot['Furniture']+dfSSPivot['Office Supplies']+dfSSPivot['Technology']).round(2)
dfSSPivot

Category,Furniture,Office Supplies,Technology,RowTotals
Segment,,,,
Consumer,391049.3120,363952.136,406399.897,1161401.34
Corporate,229019.7858,230676.462,246450.119,706146.37
Home Office,121930.6975,124418.434,183304.017,429653.15


In [ ]:
#Create the column totals, transpose the columns and rows
dfColTotals = dfSSPivot.sum().round(2).to_frame().T
dfColTotals.rename(index={0:'ColTotals'},inplace=True)
dfColTotals

Category,Furniture,Office Supplies,Technology,RowTotals
ColTotals,741999.8,719047.03,836154.03,2297200.86


In [ ]:
#dfSSPivot2 = dfSSPivot.append(dfColTotals).copy()
#dfSSPivot2

In [ ]:
dfSSPivot2 = pd.concat([dfSSPivot,dfColTotals], axis=0).copy()
dfSSPivot2

Category,Furniture,Office Supplies,Technology,RowTotals
Consumer,391049.3120,363952.136,406399.897,1161401.34
Corporate,229019.7858,230676.462,246450.119,706146.37
Home Office,121930.6975,124418.434,183304.017,429653.15
ColTotals,741999.8000,719047.030,836154.030,2297200.86


In [ ]:
#dfSSPivot2.rename(index={0:'ColTotals'},inplace=True)
#dfSSPivot2

# CrossTab | Pivot | Simpler

In [ ]:
#Create the pivot table
dfSSPivot9 = pd.pivot_table(data=dfSS,index='Segment',columns='Category',values='Sales', aggfunc='sum', margins=True).copy()
dfSSPivot9

Category,Furniture,Office Supplies,Technology,All
Segment,,,,
Consumer,391049.3120,363952.136,406399.897,1.161401e+06
Corporate,229019.7858,230676.462,246450.119,7.061464e+05
Home Office,121930.6975,124418.434,183304.017,4.296531e+05
All,741999.7953,719047.032,836154.033,2.297201e+06


In [ ]:
#pd.set_option('display.float_format', lambda x: '%.0f' % x)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

#xPlatform xCheck

In [ ]:
#Create the pivot table
dfSSPivot9 = pd.pivot_table(data=dfSS,index='Segment',columns='Category',values='Sales', aggfunc='sum', margins=True).copy()
dfSSPivot9

Category,Furniture,Office Supplies,Technology,All
Segment,,,,
Consumer,391049.31,363952.14,406399.90,1161401.34
Corporate,229019.79,230676.46,246450.12,706146.37
Home Office,121930.70,124418.43,183304.02,429653.15
All,741999.80,719047.03,836154.03,2297200.86


In [ ]:
#Create the pivot table
dfSSPivot10 = pd.pivot_table(data=dfSS,index='Category',columns='Region',values='Quantity', aggfunc='sum', margins=True).copy()
dfSSPivot10

Region,Central,East,South,West,All
Category,,,,,
Furniture,1827,2214,1291,2696,8028
Office Supplies,5409,6462,3800,7235,22906
Technology,1544,1942,1118,2335,6939
All,8780,10618,6209,12266,37873


In [ ]:
#Create the pivot table
dfSSPivot11 = pd.pivot_table(data=dfSS,index='Region',columns='Segment',values='Quantity', aggfunc='sum', margins=True).copy()
dfSSPivot11

##Pivot Tables in SQL
A cross tab with the same data is available in [this Colab Notebook](https://github.com/Praxis-QR/RDWH/blob/2d97c03428e59b1e5b9a77a4630887dc76c50d00/Cross_Tab_with_OLAP_Cube_SQL_Superstore.ipynb)

In [ ]:
from datetime import datetime
import pytz
print('Last tested  ',datetime.now(pytz.timezone('Asia/Calcutta')))

Last tested   2023-09-11 05:56:09.165962+05:30


#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself.
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)

#eCommerce Data <br>
This data has too few dimensions to be made into a pivot table <br>
Original Data https://www.kaggle.com/carrie1/ecommerce-data <br>
Same data in Google Drive https://drive.google.com/open?id=1K_rQftggePnII3K6613MumhKo3dTMO5r <br>
10% data in Google Drive https://drive.google.com/file/d/1YjCMPc4vUuXvWik1u4p5aP-THJUJjvDX/view?usp=sharing

In [ ]:
#!gdown https://drive.google.com/uc?id=1YjCMPc4vUuXvWik1u4p5aP-THJUJjvDX
!gdown -O eCommerce.csv -q 1YjCMPc4vUuXvWik1u4p5aP-THJUJjvDX

In [ ]:
#dfeCom = pd.read_csv('eCommerce02.csv', delimiter = ',')
dfeCom = pd.read_csv('eCommerce.csv', delimiter = ',')
dfeCom.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,C544414,22960,JAM MAKING SET WITH JARS,-2,2/18/2011 14:54,3.75,13408.00,United Kingdom
1,555276,48111,DOORMAT 3 SMILEY CATS,1,6/1/2011 17:28,15.79,NaN,United Kingdom
2,575656,22952,60 CAKE CASES VINTAGE CHRISTMAS,48,11/10/2011 14:29,0.55,13319.00,United Kingdom
3,571636,20674,GREEN POLKADOT BOWL,16,10/18/2011 11:41,1.25,13509.00,United Kingdom
4,576657,22556,PLASTERS IN TIN CIRCUS PARADE,12,11/16/2011 11:03,1.65,12720.00,Germany


In [ ]:
dfeCom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10838 entries, 0 to 10837
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   InvoiceNo    10838 non-null  object 
 1   StockCode    10838 non-null  object 
 2   Description  10809 non-null  object 
 3   Quantity     10838 non-null  int64  
 4   InvoiceDate  10838 non-null  object 
 5   UnitPrice    10838 non-null  float64
 6   CustomerID   8057 non-null   float64
 7   Country      10838 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 677.5+ KB


In [ ]:
#Create the pivot table
dfeComPivot = pd.pivot_table(data=dfeCom,index='StockCode',columns='Country',values='Quantity', aggfunc='sum').copy()
dfeComPivot

Country,Australia,Austria,Bahrain,Belgium,Brazil,Canada,Channel Islands,Cyprus,Czech Republic,Denmark,...,Portugal,Saudi Arabia,Singapore,Spain,Sweden,Switzerland,USA,United Arab Emirates,United Kingdom,Unspecified
StockCode,,,,,,,,,,,,,,,,,,,,,
10120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,NaN
10125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.00,NaN
10133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.00,NaN
10135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.00,NaN
11001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-176.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DCGS0069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5.00,NaN
DOT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.00,NaN
M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.00,NaN,1.00,NaN,NaN,NaN,NaN,NaN,-8.00,NaN


In [ ]:
dfeCom['Sales'] = dfeCom['UnitPrice']*dfeCom['Quantity']
dfeCom.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales
0,C544414,22960,JAM MAKING SET WITH JARS,-2,2/18/2011 14:54,3.75,13408.00,United Kingdom,-7.50
1,555276,48111,DOORMAT 3 SMILEY CATS,1,6/1/2011 17:28,15.79,NaN,United Kingdom,15.79
2,575656,22952,60 CAKE CASES VINTAGE CHRISTMAS,48,11/10/2011 14:29,0.55,13319.00,United Kingdom,26.40
3,571636,20674,GREEN POLKADOT BOWL,16,10/18/2011 11:41,1.25,13509.00,United Kingdom,20.00
4,576657,22556,PLASTERS IN TIN CIRCUS PARADE,12,11/16/2011 11:03,1.65,12720.00,Germany,19.80


In [ ]:
#dfeCom.groupby(["Country", "StockCode"], as_index=False)["Sales"].sum()
#dfeCom.groupby(["Country", "StockCode"], as_index=False)["Sales"].sum().sort_values('Sales', ascending=False)
dfeCom.groupby(["Country"], as_index=False)["Sales"].sum().sort_values('Sales', ascending=False)

,Country,Sales
33,United Kingdom,167546.57
12,France,4991.47
22,Netherlands,4880.63
10,EIRE,4147.37
13,Germany,4132.20
0,Australia,3440.75
27,Singapore,2227.19
19,Japan,1381.85
30,Switzerland,1033.54
23,Norway,954.01


In [ ]:
dfeCom.groupby(["StockCode"], as_index=False)["Sales"].sum().sort_values('Sales', ascending=False)

,StockCode,Sales
2463,DOT,10061.79
2338,85123A,5280.80
2464,M,2953.89
710,22086,2535.70
972,22423,2168.82
...,...,...
1498,23064,-149.85
947,22389,-185.34
4,11001,-215.44
2460,BANK CHARGES,-566.37


In [ ]:
tCountries = ['United Kingdom','France',	'Netherlands','EIRE','Germany','Australia','Singapore','Japan','Switzerland']
tCountries

['United Kingdom',
 'France',
 'Netherlands',
 'EIRE',
 'Germany',
 'Australia',
 'Singapore',
 'Japan',
 'Switzerland']

In [ ]:
tCodes = ['DOT','85123A','M','22086','22423','47566','22720','22178','85099B','22149','23298','23534','22326','22622']
tCodes

['DOT',
 '85123A',
 'M',
 '22086',
 '22423',
 '47566',
 '22720',
 '22178',
 '85099B',
 '22149',
 '23298',
 '23534',
 '22326',
 '22622']

In [ ]:
dfeCom.loc[dfeCom['Country'].isin(tCountries)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales
0,C544414,22960,JAM MAKING SET WITH JARS,-2,2/18/2011 14:54,3.75,13408.00,United Kingdom,-7.50
1,555276,48111,DOORMAT 3 SMILEY CATS,1,6/1/2011 17:28,15.79,NaN,United Kingdom,15.79
2,575656,22952,60 CAKE CASES VINTAGE CHRISTMAS,48,11/10/2011 14:29,0.55,13319.00,United Kingdom,26.40
3,571636,20674,GREEN POLKADOT BOWL,16,10/18/2011 11:41,1.25,13509.00,United Kingdom,20.00
4,576657,22556,PLASTERS IN TIN CIRCUS PARADE,12,11/16/2011 11:03,1.65,12720.00,Germany,19.80
...,...,...,...,...,...,...,...,...,...
10833,540468,22668,PINK BABY BUNTING,1,1/7/2011 13:55,5.91,NaN,United Kingdom,5.91
10834,544406,22616,PACK OF 12 LONDON TISSUES,1,2/18/2011 13:26,0.29,16726.00,United Kingdom,0.29
10835,559816,22231,JIGSAW TREE WITH BIRDHOUSE,4,7/12/2011 16:11,1.25,NaN,United Kingdom,5.00
10836,560716,47566,PARTY BUNTING,24,7/20/2011 13:26,4.95,17675.00,United Kingdom,118.80


In [ ]:
dfeCom.loc[dfeCom['StockCode'].isin(tCodes)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales
5,569823,23298,SPOTTY BUNTING,1,10/6/2011 12:15,4.95,16895.00,United Kingdom,4.95
20,548859,22178,VICTORIAN GLASS HANGING T-LIGHT,6,4/4/2011 13:32,1.25,13668.00,United Kingdom,7.50
34,C578073,M,Manual,-36,11/22/2011 16:02,0.56,18139.00,United Kingdom,-20.16
40,572511,23534,STOP FOR TEA WALL ART,4,10/24/2011 14:29,5.95,14895.00,United Kingdom,23.80
86,574694,22720,SET OF 3 CAKE TINS PANTRY DESIGN,6,11/6/2011 13:36,4.95,12381.00,Norway,29.70
...,...,...,...,...,...,...,...,...,...
10560,559162,23298,SPOTTY BUNTING,2,7/6/2011 16:29,10.79,NaN,United Kingdom,21.58
10584,539325,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/17/2010 8:20,2.95,13004.00,United Kingdom,17.70
10604,549262,22149,FELTCRAFT 6 FLOWER FRIENDS,1,4/7/2011 12:38,2.10,14465.00,United Kingdom,2.10
10722,555510,22178,VICTORIAN GLASS HANGING T-LIGHT,2,6/3/2011 17:09,2.46,NaN,United Kingdom,4.92


In [ ]:
dfeCom.loc[dfeCom['StockCode'].isin(tCodes) & dfeCom['Country'].isin(tCountries)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales
5,569823,23298,SPOTTY BUNTING,1,10/6/2011 12:15,4.95,16895.00,United Kingdom,4.95
20,548859,22178,VICTORIAN GLASS HANGING T-LIGHT,6,4/4/2011 13:32,1.25,13668.00,United Kingdom,7.50
34,C578073,M,Manual,-36,11/22/2011 16:02,0.56,18139.00,United Kingdom,-20.16
40,572511,23534,STOP FOR TEA WALL ART,4,10/24/2011 14:29,5.95,14895.00,United Kingdom,23.80
131,570637,23298,SPOTTY BUNTING,6,10/11/2011 12:34,4.95,12530.00,Germany,29.70
...,...,...,...,...,...,...,...,...,...
10560,559162,23298,SPOTTY BUNTING,2,7/6/2011 16:29,10.79,NaN,United Kingdom,21.58
10584,539325,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/17/2010 8:20,2.95,13004.00,United Kingdom,17.70
10604,549262,22149,FELTCRAFT 6 FLOWER FRIENDS,1,4/7/2011 12:38,2.10,14465.00,United Kingdom,2.10
10722,555510,22178,VICTORIAN GLASS HANGING T-LIGHT,2,6/3/2011 17:09,2.46,NaN,United Kingdom,4.92


In [ ]:
dfeComR = dfeCom.loc[dfeCom['StockCode'].isin(tCodes) & dfeCom['Country'].isin(tCountries)].copy()
dfeComR

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales
5,569823,23298,SPOTTY BUNTING,1,10/6/2011 12:15,4.95,16895.00,United Kingdom,4.95
20,548859,22178,VICTORIAN GLASS HANGING T-LIGHT,6,4/4/2011 13:32,1.25,13668.00,United Kingdom,7.50
34,C578073,M,Manual,-36,11/22/2011 16:02,0.56,18139.00,United Kingdom,-20.16
40,572511,23534,STOP FOR TEA WALL ART,4,10/24/2011 14:29,5.95,14895.00,United Kingdom,23.80
131,570637,23298,SPOTTY BUNTING,6,10/11/2011 12:34,4.95,12530.00,Germany,29.70
...,...,...,...,...,...,...,...,...,...
10560,559162,23298,SPOTTY BUNTING,2,7/6/2011 16:29,10.79,NaN,United Kingdom,21.58
10584,539325,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/17/2010 8:20,2.95,13004.00,United Kingdom,17.70
10604,549262,22149,FELTCRAFT 6 FLOWER FRIENDS,1,4/7/2011 12:38,2.10,14465.00,United Kingdom,2.10
10722,555510,22178,VICTORIAN GLASS HANGING T-LIGHT,2,6/3/2011 17:09,2.46,NaN,United Kingdom,4.92


In [ ]:
dfeComPivot = pd.pivot_table(data=dfeComR,index='StockCode',columns='Country',values='Sales', aggfunc='sum').copy()
dfeComPivot

Country,Australia,EIRE,France,Germany,Netherlands,Singapore,Switzerland,United Kingdom
StockCode,,,,,,,,
22086,NaN,NaN,NaN,NaN,306.00,NaN,NaN,2229.70
22149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1214.58
22178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1438.32
22326,NaN,NaN,17.70,88.50,734.40,NaN,NaN,64.80
22423,700.80,NaN,NaN,NaN,NaN,NaN,175.20,859.32
22622,NaN,23.90,NaN,NaN,NaN,NaN,NaN,952.63
22720,1020.00,204.00,NaN,14.85,NaN,NaN,NaN,584.38
23298,NaN,NaN,NaN,29.70,NaN,NaN,44.55,1133.53
23534,963.00,NaN,NaN,NaN,NaN,NaN,NaN,84.42
